# Kipindi cha 6 – Njia ya Hatua Nyingi (Panga → Tekeleza → Rekebisha)

Inatumia kazi za uelekezaji kuchagua mifano kwa kila hatua, kisha inarekebisha jibu la mwisho.


### Maelezo: Usakinishaji wa Vitegemezi
Inasakinisha `foundry-local-sdk` na `openai` zinazohitajika kwa miito ya mazungumzo ya hatua kwa hatua. Ni salama kuendesha tena.


# Hali
Mfumo wa hatua nyingi wa mfano ambao: (1) Unapanga kazi katika hatua tofauti, (2) Unatekeleza kila hatua kwa kuchagua mfano kulingana na nia, (3) Unaboresha jibu la mwisho lililosanidiwa. Unaonyesha kuunganisha uwezo tofauti wa SLM kwa ufanisi huku ukihifadhi ubora.


In [13]:
!pip install -q foundry-local-sdk openai

### Maelezo: Uingizaji Muhimu
Inaingiza regex kwa utambuzi wa nia, Foundry Local manager kwa usimamizi wa viambatisho vya kila alias, na mteja wa OpenAI kwa kukamilisha mazungumzo.


In [14]:
import re
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Maelezo: Katalogi ya Uwezo na Sheria  
Inafafanua katalogi inayotambua uwezo na sheria za regex zinazotumika kuoanisha maandishi ya hatua na kategoria za nia (kodi, muhtasari, uainishaji, jumla). Thamani za kipaumbele cha chini hushinda pale ambapo mifano mingi inaunga mkono nia moja.


In [15]:
CATALOG = {
 'phi-4-mini': {'capabilities':['general','summarize'],'priority':2},
 'qwen2.5-coder-7b': {'capabilities':['code','refactor'],'priority':1},
 'qwen2.5-0.5b': {'capabilities':['classification','fast'],'priority':3},
}
RULES = [
 (re.compile('code|refactor|function', re.I), 'code'),
 (re.compile('summari|abstract|tl;dr', re.I), 'summarize'),
 (re.compile('classif|category|label', re.I), 'classification'),
]

### Maelezo: Nia, Uchaguzi wa Mfano & Msaidizi wa Gumzo
Inatoa:
- `detect_intent` kwa uainishaji unaotegemea regex.
- `pick_model` kuchagua alias bora kulingana na uwezo + kipaumbele.
- `chat` kifungashio cha urahisi kinachounda mteja kwa kila alias na kurudisha jibu la mzunguko mmoja.


In [16]:
def detect_intent(text: str):
    """Return an intent label based on simple regex rules; falls back to 'general'."""
    for pat, intent in RULES:
        if pat.search(text):
            return intent
    return 'general'

def pick_model(intent: str) -> str:
    """Pick the best model for an intent using capability match first, then priority."""
    ranked = []
    for name, meta in CATALOG.items():
        ranked.append((name, intent in meta['capabilities'], meta['priority']))
    # Sort: capability match (True first), then lower priority value
    ranked.sort(key=lambda t: (not t[1], t[2]))
    return ranked[0][0]

def chat(alias: str, content: str, temp: float = 0.4) -> str:
    """Simple helper to send a single user message to a Foundry Local model via OpenAI client."""
    m = FoundryLocalManager(alias)
    client = OpenAI(base_url=m.endpoint, api_key=m.api_key or 'not-needed')
    mid = m.get_model_info(alias).id
    resp = client.chat.completions.create(
        model=mid,
        messages=[{'role': 'user', 'content': content}],
        max_tokens=180,
        temperature=temp,
    )
    return resp.choices[0].message.content

### Maelezo: Kazi ya Mchakato wa Hatua Nyingi
Inatekeleza mchakato: kupanga → kuchambua hatua → kutekeleza kila moja kwa kutumia njia ya kuelekeza kwa nia → kuboresha matokeo yaliyounganishwa. Inarudisha kamusi iliyopangwa kwa ukaguzi au tathmini.


In [17]:
def pipeline(task: str):
    """Multi-step pipeline: plan → execute steps → refine final answer.

    Returns dict with keys: plan (raw plan text), steps (list of tuples), final (refined answer).
    Each step tuple: (index, step_text, step_result, model_alias_used)
    """
    # 1. Plan
    plan_alias = pick_model('general')
    plan_prompt = (
        "Break the task into 3 concise, actionable steps (no extra commentary).\n"
        f"Task: {task}"
    )
    plan = chat(plan_alias, plan_prompt)

    # 2. Parse steps (robust to numbering or bullet styles)
    raw_lines = [l.strip() for l in plan.splitlines() if l.strip()]
    steps = []
    for line in raw_lines:
        cleaned = re.sub(r'^\d+[).:-]?\s*', '', line)  # remove leading numbering
        cleaned = re.sub(r'^[-*]\s*', '', cleaned)      # remove bullet markers
        if cleaned:
            steps.append(cleaned)
        if len(steps) == 3:
            break
    if not steps:
        steps = [task]

    # 3. Execute steps
    outputs = []
    for idx, step in enumerate(steps, 1):
        intent = detect_intent(step)
        exec_alias = pick_model(intent)
        exec_prompt = (
            f"Execute step {idx} for the overall task.\n"
            f"Overall task: {task}\n"
            f"Step {idx}: {step}\n"
            "Return a concise result focusing only on the step objective."
        )
        result = chat(exec_alias, exec_prompt)
        outputs.append((idx, step, result, exec_alias))

    # 4. Refine final answer
    refine_alias = pick_model('summarize')
    combined = "\n\n".join(
        f"Step {idx} ({alias}) Output:\n{res}" for idx, _step, res, alias in outputs
    )
    refine_prompt = (
        "You are a senior assistant. Synthesize these step outputs into a cohesive final answer.\n"
        "Ensure clarity, avoid repetition, and highlight improvements or key insights if relevant.\n\n"
        f"Task: {task}\n\n"
        f"Step Outputs:\n{combined}"
    )
    final_ans = chat(refine_alias, refine_prompt)

    return {"plan": plan, "steps": outputs, "final": final_ans}

### Maelezo: Endesha Kazi ya Mfano  
Inatekeleza mchakato mzima kwenye kazi inayolenga kufanyia marekebisho ikionyesha nia mchanganyiko (kodi + muhtasari). Kamusi ya matokeo inaonyesha mpango wa awali, matokeo ya kila hatua na majina ya mifano iliyochaguliwa, na jibu la mwisho lililounganishwa.


In [18]:
result = pipeline('Generate a refactored version of a slow Python loop and summarize performance gains.')
result

{'plan': '1. Profile the existing slow Python loop to identify bottlenecks.\n2. Refactor the loop using optimized techniques (e.g., list comprehensions, map, or itertools).\n3. Compare the performance of the refactored loop with the original using a benchmarking tool and summarize the gains.',
 'steps': [(1,
   'Profile the existing slow Python loop to identify bottlenecks.',
   "To execute step 1, you would use a profiling tool like `cProfile` in Python to analyze the performance of the existing slow loop. Here's an example of how you might do this:\n\n```python\nimport cProfile\n\ndef slow_loop():\n    # Example of a slow loop\n    result = []\n    for i in range(1000000):\n        result.append(i * i)\n    return result\n\n# Profile the slow loop\ncProfile.run('slow_loop()', 'profile_stats')\n\n# To see the results, you can use pstats module\nimport pstats\np = pstats.Stats('profile_stats')\np.sort_stats('cumulative').print_stats()\n```\n\nThis code will run the `slow_loop` function

### Maelezo: Onyesha Kitu cha Matokeo
Inaonyesha matokeo ya muundo wa bomba kwa ukaguzi wa haraka au tathmini ya hatua inayofuata (mfano, kupima ubora wa hatua au ufanisi wa uboreshaji).



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
